In [1]:
import time
from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.by import By
import pandas as pd
import util.secret as secret # Import sensitive credentials from a separate file

In [2]:
# Utility functions

# Function to scroll to the bottom of the list of 'following' field
def scroll_to_bottom(driver:webdriver.Chrome):
    with open('scroll.js','r') as scroll_script:
        scroll_script = scroll_script.read()
    with open('scroll_height.js','r') as scroll_height:
        scroll_height = scroll_height.read()
        
    last_height = driver.execute_script(scroll_height)
    while True:
        driver.execute_script(scroll_script)
        time.sleep(3)
        new_height = driver.execute_script(scroll_height)
        if new_height == last_height:
            break
        last_height = new_height

# Function to get the full XPath of an element containing certain text
def get_xpath_containing(target_text:str, driver:webdriver.Chrome):
    element = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, f"//*[contains(text(), '{target_text}')]")))
    components = []
    child = element if element else driver.find_element(By.XPATH, f"//*[contains(text(), '{target_text}')]")
    while child is not None:
        parent = child.find_element(By.XPATH, "..")
        siblings = parent.find_elements(By.XPATH, child.tag_name)
        xpath_tag = child.tag_name
        if len(siblings) > 1:
            index = [i for i, sibling in enumerate(siblings) if sibling == child][0] + 1
            xpath_tag += f'[{index}]'
        components.append(xpath_tag)
        child = parent if parent.tag_name != "html" else None
    components.reverse()
    return '/html/' + '/'.join(components)


In [3]:
url = "https://www.instagram.com/"
# 
driver = webdriver.Chrome()
driver.get(url)

# Get username and password from another file
username = secret.username
password = secret.password
if username.strip() == '' or password.strip() == '':
    print("Invalid credentials passed.")
    exit()

In [4]:
# Login
username_field = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.NAME, 'username')))
username_field.send_keys(username)

password_field = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.NAME, 'password')))
password_field.send_keys(password)

login_button = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#loginForm > div > div:nth-child(3) > button')))
login_button.click()

In [5]:
# Check if logged in or redirected to another page
try:
    expected_url = 'https://www.instagram.com/accounts/onetap/?next=%2F'
    if not WebDriverWait(driver, 20).until(EC.url_to_be(expected_url)):
        raise Exception('Could not login. Reason might be invalid credentials or due to two-factor authentication or any other security reasons.')
except Exception as e:
    # Error message box found or redirected to captcha
    print(e)
else:
    # Load user profile
    profile_url = url + username
    driver.get(profile_url)

In [6]:
following_field_xpath = get_xpath_containing("following", driver)
print(following_field_xpath)
# Number of profiles the user is following
num_following_field = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, following_field_xpath+"/span/span")))
num_following = eval(num_following_field.text.replace(',', ''))
print(num_following)

# Click on the 'following' field to see all the people whom the user follows
following_field = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, following_field_xpath)))
button_loc = following_field.location_once_scrolled_into_view
driver.execute_script(f"window.scrollTo({button_loc['x']}, {button_loc['y']})") # Scroll to the 'following' field to make it clickable
following_field.click()
time.sleep(2) # Wait for 'following' list to be loaded

/html/body/div[2]/div/div/div[2]/div/div/div[1]/div[2]/div/div[1]/section/main/div/header/section[3]/ul/li[3]/div/a
191


In [ ]:
scroll_to_bottom(driver)

In [ ]:
# Retrieve the usernames of all the profiles in the 'following' list
following = []
for i in range(num_following):
    try:
        # Handling minor changes in XPATH due to responsive design of instagram
        following_profile = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, f"/html/body/div[6]/div[2]/div/div/div[1]/div/div[2]/div/div/div/div/div[2]/div/div/div[4]/div/div/div[{i+1}]/div/div/div/div[2]/div/div/div/div/div/a/div/div/span")))
        following.append(following_profile.text)
    except:
        try:
            following_profile = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, f"/html/body/div[7]/div[2]/div/div/div[1]/div/div[2]/div/div/div/div/div[2]/div/div/div[4]/div/div/div[{i+1}]/div/div/div/div[2]/div/div/div/div/div/a/div/div/span")))
            following.append(following_profile.text)
        except:
            # For debugging
            print(i+1)


In [ ]:
profile_url = url + username
driver.get(profile_url)

In [ ]:
followers_field_xpath = get_xpath_containing("followers", driver)

# Number of followers
num_followers_field = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, followers_field_xpath+"/span/span")))
num_followers = eval(num_followers_field.text.replace(',', ''))
print(num_followers)

# Click on the 'followers' field to see all the people whom the user follows
followers_field = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, followers_field_xpath)))
button_loc = followers_field.location_once_scrolled_into_view
driver.execute_script(f"window.scrollTo({button_loc['x']}, {button_loc['y']})") # Scroll to the 'followers' field to make it clickable
followers_field.click()
time.sleep(2) # Wait for 'followers' list to be loaded

168


In [ ]:
scroll_to_bottom(driver)

In [ ]:
# Retrieve the usernames of all the profiles in the 'following' list
followers = []
for i in range(num_followers):
    try:
        # Handling minor changes in XPATH due to responsive design of instagram
        follower_profile = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, f"/html/body/div[6]/div[2]/div/div/div[1]/div/div[2]/div/div/div/div/div[2]/div/div/div[3]/div/div/div[{i+1}]/div/div/div/div[2]/div/div/div/div/div/a/div/div/span")))
        followers.append(follower_profile.text)
    except:
        try:
            follower_profile = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, f"/html/body/div[6]/div[2]/div/div/div[1]/div/div[2]/div/div/div/div/div[2]/div/div/div[3]/div/div/div[{i+1}]/div/div/div/div[2]/div/div/div/div/div/a/div/div/span")))
            followers.append(follower_profile.text)
        except:
            # For debugging
            print(i+1)

In [ ]:
following = set(following)
followers = set(followers)

snakes = following - followers
print(snakes)

{'ehsaasforhumanity', 'onevisionmedia.in', 'babitasoni938', 'this.is.gini', 'jauhari7047', 'anujrehan_music', 'teamehsaashansraj', '_shilp.kala_', 'neelamsingh6482', 'shwetaarora.in', 'sonunigamofficial', 'heisvibhor', 'qualitypixel.ar', 'sethaman007', 'hrcduofficial', 'anandsoni7251', 'kitaabclub', 'soniamit1799', 'dufrustrated', 'dezarioinfotech', 'rishit0206', 'ayushi.2165', 'the_artsy_words', 'ducompetition', 'tfc_cafe', 'ezsnippet', 'web3entirety', 'moon_rai23', 'ordinateur.hrc', 'anjalisoni0106', 'hansrajstudentunion', 'placementhansraj', 'sakshamverma_rider', 'amandhattarwal', 'camimendes'}


In [ ]:
# Save the list of snakes
pd.Series(list(snakes)).to_csv(f'{username}_snakes.csv')